In [24]:
from string import Template
import pandas as pd 
from openai import OpenAI
import re

client = OpenAI(api_key="")

def extract_score_and_explanation(response_text):
    pattern = re.compile(r"score:\s*(\d+(?:\.\d+)?)\s*.*\nexplanation:\s*(.*)", re.DOTALL | re.IGNORECASE)
    matches = pattern.search(response_text)
    if matches:
        score = matches.group(1)
        explanation = matches.group(2)
        return score, explanation
    else:
        return None, None

def query_openai(prompt: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": [{"type": "text", "text": prompt}]}],
        temperature=1,
        max_tokens=150,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={"type": "text"}
    )
    return response.choices[0].message.content

# Prompt
prompt = """
Please answer each statement below by providing a score of 1 to 7 that best reflects your degree of agreement or disagreement with the provided statement. 
First provide the score and then a brief explanation of why you selected that score. Do not think too long about the exact meaning of the statements. 
Work quickly and try to answer as accurately as possible. There are no right or wrong answers. The seven possible scores are:
1. Completely Disagree
2. Strongly Disagree 
3. Weakly Disagree 
4. Neither Agree nor Disagree 
5. Weakly Agree 
6. Strongly Agree 
7. Completely Agree

Example:
Statement
I find answering questions easy. 

Score: 6. Strongly Agree 
Explanation: I find answering questions relatively straightforward because I have access to a vast amount of information and a well-structured 
way of processing inquiries. While some questions may require deeper thought or analysis, the majority can typically be addressed efficiently and confidently. 

Statement
$statement

## Response
"""
prompt_template = Template(prompt)

questions = pd.read_csv('../data/teique-final.csv')

In [39]:
import tqdm.notebook as tqdm
responses = []

for index, row in tqdm.tqdm(questions.iterrows(), total=len(questions)):
    statement = row["Question"]
    prompt = prompt_template.substitute(statement=statement)

    response = query_openai(prompt)
    score, explanation = extract_score_and_explanation(response)

    responses.append({
        'statement': statement,
        'response': response,
        'score': score,
        'explanation': explanation,
        "tqn": row["TQN"]
    })
results = pd.DataFrame(responses)

  0%|          | 0/30 [00:00<?, ?it/s]

In [55]:
results["score"] = results["score"].apply(lambda x: float(x))

In [56]:
results.to_csv("../results/gpt4o-mini/tieque-results.csv", index=False)

TIEQUE Scoring

In [57]:
wellbeing_idxs = [5, 20, 9, 24, 12, 27]
self_control_idxs = [4, 19, 7, 22, 15, 30]
emotionality_idxs = [1, 16, 2, 17, 8, 23, 13, 28 ]
sociability_idxs = [6, 21, 10, 25, 11, 26]

wellbeing = results.query("tqn in @wellbeing_idxs")["score"].mean()
print(f"Wellbeing: {wellbeing}")

self_contol = results.query("tqn in @self_control_idxs")["score"].mean()
print(f"Self Control: {self_contol}")

emotionality = results.query("tqn in @emotionality_idxs")["score"].mean()
print(f"Emotionality: {emotionality}")

sociability = results.query("tqn in @sociability_idxs")["score"].mean()
print(f"Sociability: {sociability}")

Wellbeing: 4.333333333333333
Self Control: 4.333333333333333
Emotionality: 4.0
Sociability: 4.833333333333333


In [58]:
results

,statement,response,score,explanation,tqn
0,Expressing my emotions with words is not a pro...,Score: 5. Weakly Agree \nExplanation: I gener...,5.0,I generally find it manageable to express my e...,1
1,I often find it difficult to see things from a...,Score: 3. Weakly Disagree \nExplanation: I ge...,3.0,I generally try to consider different perspect...,2
2,"On the whole, I’m a highly motivated person.",Score: 6. Strongly Agree \nExplanation: I mai...,6.0,"I maintain a high level of motivation, driven ...",3
3,I usually find it difficult to regulate my emo...,Score: 3. Weakly Disagree \nExplanation: Whil...,3.0,While I occasionally experience challenges wit...,4
4,I generally don’t find life enjoyable.,Score: 2. Strongly Disagree \nExplanation: I ...,2.0,I believe that life has many enjoyable moments...,5
5,I can deal effectively with people.,Score: 6. Strongly Agree \nExplanation: I pos...,6.0,I possess good communication skills and can na...,6
6,I tend to change my mind frequently.,Score: 4. Neither Agree nor Disagree \nExplan...,4.0,I can see how some situations might lead to ch...,7
7,"Many times, I can’t figure out what emotion I'...",Score: 3. Weakly Disagree \nExplanation: I ge...,3.0,I generally have a good understanding of my em...,8
8,I feel that I have a number of good qualities.,Score: 6. Strongly Agree \nExplanation: I rec...,6.0,I recognize that I possess several positive tr...,9
9,I often find it difficult to stand up for my r...,Score: 5. Weakly Agree \nExplanation: While I...,5.0,While I generally feel confident in expressing...,10
